In [12]:
%matplotlib inline


Extract Feature for Shapley Calculation
=============================

**Adopted from:** [Official PyTorch tutorial on fine-tuning torchvision models](https://pytorch.org/tutorials/beginner/finetuning_torchvision_models_tutorial.html)




In [13]:
from __future__ import print_function 
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

import pickle
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score
from collections import Counter, defaultdict

print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)


PyTorch Version:  1.4.0
Torchvision Version:  0.5.0


In [14]:
data_dir = '/data/GQA/Perspective-Merged'
model_name = "resnet"
num_classes = 2
batch_size = 64
num_epochs = 1
feature_extract = True 

In [15]:
def extract_model_features(model, dataloaders, criterion, optimizer, num_epochs=25, is_inception=False):
    since = time.time()

    val_acc_history = []
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val', 'shapley_val',]:
            model.eval()   # Set model to evaluate mode, for extraction

            ##################################
            # Fields to be stored for postprocessing 
            ##################################
            feature_all = []
            target_all = []

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # forward
                # track history if only in train
                with torch.set_grad_enabled(False):
                    outputs  = model(inputs)
                    features = outputs
                    # _, preds = torch.max(outputs, 1)
                    with torch.no_grad():
                        ##################################
                        # Evaluation book-keeping Field 
                        ##################################
                        feature_all.append( outputs.cpu().numpy() )
                        target_all.append( labels.cpu().numpy() )

            ##################################
            # Evaluation book-keeping Field 
            ##################################
            target_all     = np.concatenate( target_all, axis=0)
            feature_all     = np.concatenate( feature_all, axis=0)
            print('feature_all', feature_all.shape)

            dump_result_dict = {
                "target_all": target_all, 
                "feature_all": feature_all, 
                }
            with open(os.path.join('outputs', 'feature_dump_{}.pkl'.format(phase) ), "wb") as pkl_file:
                pickle.dump(
                    dump_result_dict, 
                    pkl_file, 
                )

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

In [16]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [17]:
def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    model_ft = None
    input_size = 0

    if model_name == "resnet":
        """ Resnet18
        """
        model_ft = models.resnet18(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Identity() # nn.Linear(num_ftrs, num_classes)
        input_size = 224

    else:
        print("Invalid model name, exiting...")
        exit()
    
    return model_ft, input_size

# Initialize the model for this run
model_ft, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=True)

# Print the model we just instantiated
# print(model_ft)

In [18]:
# Data augmentation and normalization for training
# Just normalization for validation
extract_transform = transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

print("Initializing Datasets and Dataloaders...")

# Create training and validation datasets
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), extract_transform) for x in ['train', 'val', 'shapley_val']}
# Create training and validation dataloaders
dataloaders_dict = {
    'train': torch.utils.data.DataLoader(image_datasets['train'], batch_size=batch_size, shuffle=False, num_workers=4), 
    'val'  : torch.utils.data.DataLoader(image_datasets['val'], batch_size=batch_size, shuffle=False, num_workers=4),
    'shapley_val'  : torch.utils.data.DataLoader(image_datasets['shapley_val'], batch_size=batch_size, shuffle=False, num_workers=4),
}
# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Initializing Datasets and Dataloaders...


In [19]:
# Send the model to GPU
model_ft = model_ft.to(device)
optimizer_ft = None

Run Training and Validation Step
--------------------------------

Finally, the last step is to setup the loss for the model, then run the
training and validation function for the set number of epochs. Notice,
depending on the number of epochs this step may take a while on a CPU.
Also, the default learning rate is not optimal for all of the models, so
to achieve maximum accuracy it would be necessary to tune for each model
separately.




In [20]:
# Setup the loss fxn
criterion = nn.CrossEntropyLoss()

# Train and evaluate
model_ft, hist = extract_model_features(model_ft, dataloaders_dict, criterion, optimizer_ft, num_epochs=num_epochs, is_inception=(model_name=="inception"))

Epoch 0/0
----------
feature_all (1500, 512)
feature_all (240, 512)
feature_all (238, 512)

Training complete in 0m 7s
Best val Acc: 0.000000


In [21]:
image_datasets['val']

Dataset ImageFolder
    Number of datapoints: 240
    Root location: /data/GQA/Perspective-Merged/val
    StandardTransform
Transform: Compose(
               Resize(size=224, interpolation=PIL.Image.BILINEAR)
               CenterCrop(size=(224, 224))
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )

In [22]:
image_datasets['shapley_val']

Dataset ImageFolder
    Number of datapoints: 238
    Root location: /data/GQA/Perspective-Merged/shapley_val
    StandardTransform
Transform: Compose(
               Resize(size=224, interpolation=PIL.Image.BILINEAR)
               CenterCrop(size=(224, 224))
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )